In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from pytube import YouTube

import random, time

import config

In [ ]:
service = Service(config.WEBDRIVER_PATH)

browser_options = webdriver.ChromeOptions()
options = [
    '--disable-blink-features', 
    '--no-sandbox', 
    '--disable-extensions',
    '--ignore-certificate-errors', 
    #'--headless',
    '--window-size=1920,1080'
   ]
for option in options:
    browser_options.add_argument(option)
browser = webdriver.Chrome(service=service, options=browser_options)


searchQuery = "config.SEARQUERY"

youtube = "https://www.youtube.com"

In [ ]:
def get_website():
    browser.get(youtube)
    time.sleep(10)
    
    try:
        browser.find_element(By.XPATH, "/html/body/ytd-app/ytd-consent-bump-v2-lightbox/tp-yt-paper-dialog/div[4]/div[2]/div[6]/div[1]/ytd-button-renderer[2]/a/tp-yt-paper-button").click()
        print("first xpath correct")
    except NoSuchElementException :
        print("cookie button non responsive changing xpath...")
        time.sleep(3)
        key= browser.find_element(By.XPATH, "/html/body/ytd-app/ytd-consent-bump-v2-lightbox/tp-yt-paper-dialog/div[4]/div[2]/div[6]/div[1]/ytd-button-renderer[2]/yt-button-shape/button").click()

In [ ]:
def get_results():
    browser.find_element(By.XPATH, "//input[@id='search']").send_keys(searchQuery)
    browser.find_element(By.XPATH, "//button[@id='search-icon-legacy']").click()
    time.sleep(3)
    browser.find_element(By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div[2]/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer").click()
    browser.find_element(By.XPATH, "//div[@title='Search for Video']").click()
    print('filtering for video format...')
    time.sleep(3)
    browser.find_element(By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div[2]/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer").click()
    browser.find_element(By.XPATH, "//div[@title='Search for 4 - 20 minutes']").click()
    print("filtering for between 4 to 20 min length...")


In [ ]:
def fetch_results():
    fetch_elements =  browser.find_elements(By.TAG_NAME, 'ytd-video-renderer')
    return fetch_elements

In [ ]:
def fetch_list():
    count = 0
    rangeCount = 0
    f =  open(str(searchQuery)+".txt", "w+")
    lines = f.readlines()
    for i in range(400):
        rangeCount +=1
        print(rangeCount)
        time.sleep(2) 
        browser.execute_script('window.scrollBy(0,1600)', '')
        try:
            browser.find_element(By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div[2]/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[31]/div[3]/ytd-message-renderer/yt-formatted-string[1]")
            results = fetch_results()
            total = len(results)
            print(f"total video found: {total}")
            for r in results:
                count +=1
                link = r.find_element(By.TAG_NAME, 'a').get_attribute('href')
                if link in [line for line in lines]:
                    pass
                else:
                    f.write(link+ '\n')
                    yt = YouTube(link)
                    yt.streams.get_highest_resolution().download(config.SAVE_PATH)
                    print(f"count at {count} with link {link}")
        except NoSuchElementException:
            continue
        

In [ ]:
get_website()
time.sleep(5)
get_results()
time.sleep(5)
fetch_list()